# Laboratory values vs outcome

Get the first laboratory measurements for patients admitted to the ICU. Plot the distribution of measurements for survival and non-survival groups.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from scipy.stats import ks_2samp
%matplotlib inline
plt.style.use('ggplot') 

## Get first laboratory measurement on ICU admission

In [ ]:
# create a database connection
sqluser = 'group16'
dbname = 'eicu'
sqlhost = 'sl-us-dal-9-portal.6.dblayer.com'
sqlport = 20689

query_schema = 'set search_path to eicu;'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=sqlhost, port=sqlport)

# Modify first day labs code: 
# https://github.com/MIT-LCP/mimic-code/tree/master/etc/firstday
query = query_schema + \
"""
select patientunitstayid
, acutephysiologyscore as apsiii
, 1 / (1 + exp(- (-4.4360 + 0.04726*(acutephysiologyscore) ))) as APSIII_PROB
, apachescore
, actualhospitalmortality
from apachepatientresult
where apacheversion = 'IVa';
"""

eicu = pd.read_sql_query(query,con)
con.close()
eicu.head()

In [ ]:
az = pd.read_csv('Datathon_2006_2015_CORE_APD_clean.csv')
az.head()

In [ ]:
az['apsiii'] = \
az['albuminscore'] + \
az['bilirubinscore'] + \
az['creatininescore'] + \
az['glucosescore'] + \
az['haematocritscore'] + \
az['heartratescore'] + \
az['meanarterialpressurescore'] + \
az['sodiumscore'] + \
az['neurologicalscore'] + \
az['oxygenationscore'] + \
az['phscore'] + \
az['respiratoryratescore'] + \
az['temperaturescore'] + \
az['ureascore'] + \
az['urineoutputscore'] + \
az['whitecellcountscore']

az['apsiii_prob'] = 1 / (1 + np.exp(- (-4.4360 + 0.04726*(az['apsiii']) )))

In [ ]:
# create a database connection for mimic
sqluser = 'alistairewj'
dbname = 'mimic'
sqlhost = 'localhost'
sqlport = 5432

query_schema = 'set search_path to public,mimiciii;'

con_mimic = psycopg2.connect(dbname=dbname, user=sqluser, host=sqlhost, port=sqlport)

# Modify first day labs code: 
# https://github.com/MIT-LCP/mimic-code/tree/master/etc/firstday
query = query_schema + \
"""
with t1 as
(
select
    icustay_id
    , ROW_NUMBER() over (partition by ie.subject_id order by intime) as rn
from icustays ie
inner join patients pt
on ie.subject_id = pt.subject_id
and pt.dob < (ie.intime - interval '1' year)
)
select icustay_id
, apsiii
, APSIII_PROB
, adm.hospital_expire_flag as hosdead
from apsiii ap
inner join admissions adm
on ap.hadm_id = adm.hadm_id
where icustay_id in
(select icustay_id from t1 where rn = 1);
"""

mimic = pd.read_sql_query(query,con_mimic)
con_mimic.close()
mimic.head()

In [ ]:

# define hospital mortality
az['hosdead'] = (az['hosp_outcm']==2).astype(float)
eicu['hosdead'] = (eicu['actualhospitalmortality']=='EXPIRED').astype(float)

idx = (az['admepisode'] < 2)  & (az['apache3isincluded']==1) & (az['apache3score']>0)
az_filt = az.loc[idx, ['apache3riskofdeath','apache3score',
                       'apsiii','apsiii_prob','apache3isincluded','hosdead']]
az_filt.head()

# stack data together
az_filt['db'] = 'anzics'
mimic['db'] = 'mimic'
eicu['db'] = 'eicu'


df_all = pd.concat( [ az_filt[['apsiii','apsiii_prob','hosdead','db']],
                     mimic[['apsiii','apsiii_prob','hosdead','db']],
                     eicu[['apsiii','apsiii_prob','hosdead','db']] ]
                   , axis=0, ignore_index=True)

In [ ]:
bins = np.linspace(0, 200, 101)
plt.figure(figsize=[16,10])
#plt.hist(df_all.loc[df_all['db']=='mimic','apsiii'].values, bins,
#         normed=True, alpha=0.5, label='mimic')
plt.hist(df_all.loc[df_all['db']=='eicu','apsiii'].values, bins,
         normed=True, alpha=0.5, label='eicu')
plt.hist(df_all.loc[df_all['db']=='anzics','apsiii'].values, bins,
         normed=True, alpha=0.5, label='anzics')
plt.legend(loc='upper right')
plt.show()

As we can see the distributions are very similar, though ANZICS seems to have lower acuity.

In [ ]:
bins = np.linspace(0, 200, 101)
plt.figure(figsize=[16,10])
plt.rcParams.update({'font.size': 20})
plt.hist(df_all.loc[df_all['db']=='mimic','apsiii'].values, bins,
         normed=True, alpha=0.2, label='mimic')
plt.hist(df_all.loc[df_all['db']=='eicu','apsiii'].values, bins,
         normed=True, alpha=0.5, label='eicu')
plt.hist(df_all.loc[df_all['db']=='anzics','apsiii'].values, bins,
         normed=True, alpha=0.5, label='anzics')
plt.legend(loc='upper right')
plt.show()

In [ ]:
bins = np.linspace(0, 200, 101)
plt.figure(figsize=[16,10])
plt.rcParams.update({'font.size': 20})
plt.hist(df_all.loc[df_all['db']=='mimic','apsiii'].values, bins,
         normed=True, alpha=1, linewidth=4, cumulative=True, label='mimic', histtype='step')
plt.hist(df_all.loc[df_all['db']=='eicu','apsiii'].values, bins,
         normed=True, alpha=1, linewidth=4, cumulative=True, label='eicu', histtype='step')
plt.hist(df_all.loc[df_all['db']=='anzics','apsiii'].values, bins,
         normed=True, alpha=1, linewidth=4, cumulative=True, label='anzics', histtype='step')
plt.legend(loc='upper right')
plt.show()

In [ ]:
bins = np.linspace(0, 200, 101)
plt.figure(figsize=[16,10])
plt.rcParams.update({'font.size': 20})
plt.hist(df_all.loc[df_all['db']=='mimic','apsiii'].values, bins,
         normed=True, alpha=1.0, label='mimic')
plt.hist(df_all.loc[df_all['db']=='eicu','apsiii'].values, bins,
         normed=True, alpha=0.5, label='eicu')
plt.hist(df_all.loc[df_all['db']=='anzics','apsiii'].values, bins,
         normed=True, alpha=0.5, label='anzics')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# calibration curve
xi = np.linspace(0,1,11)

calib = dict()
xi_all = dict()

db = ['anzics','eicu','mimic']

for d in db:
    calib[d] = np.zeros(len(xi)-1)
    xi_all[d] = np.zeros(len(xi)-1)
    dat = df_all.loc[df_all['db']==d,:]
    for i,x in enumerate(xi[1:]):
        idx = (dat['apsiii_prob']>xi[i])&(dat['apsiii_prob']<=xi[i+1])
        xi_all[d][i] = dat.loc[idx,'hosdead'].mean()
        calib[d][i] = dat.loc[idx,'apsiii_prob'].mean()


In [ ]:
plt.figure(figsize=[16,10])
plt.rcParams.update({'font.size': 20})
markers = ['o','d','^']

for i,d in enumerate(db):
    plt.plot(xi_all[d], calib[d], '--', marker=markers[i], label=d)
    
plt.plot([0,1],[0,1],'k-',linewidth=3)
plt.xlim([0,1])
plt.ylim([0,1])
plt.legend(loc='lower right')
plt.xlabel('Observed mortality')
plt.ylabel('Expected (predicted) mortality')
plt.show()